In [1]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

In [2]:
# Création du dictionnaire d'NFT
dico_nft = {}

for i in range(0, 100):
    nftId = f"NFT{i}"
    dico_nft[nftId] = f"../nft_images/{nftId}.png"

In [3]:
# Import des modules nécessaires
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake
from smart_contract import SmartContractDefinition, SmartContractWritingOperation

# Création de plusieurs wallets pour les nœuds de la blockchain
walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

# Création d'un wallet propriétaire pour le smart contract
walletProprietaire = Wallet()

# Création de wallets user :
walletMatthieu = Wallet()
walletEdouard = Wallet()
walletMilo = Wallet()
walletHugo = Wallet()
walletVincent = Wallet()
walletHenri = Wallet()

# Initialisation du mécanisme de Proof of Stake avec le premier nœud
pos = ProofOfStake(walletNode1.publicKey)

# On reset le network
Node.reset_network()

# On connecte les 3 noeuds au network
node1 = BlockchainNode(walletNode1,pos)
node2 = BlockchainNode(walletNode2,pos)
node3 = BlockchainNode(walletNode3,pos)

assert len(Node.get_all_nodes()) == 3
"Success"

'Success'

In [4]:
# Exemple de source de code pour le smart contract
source_code = """
class MyNFTContract:
    def __init__(self, issuerPublicKey):
        self.issuerPublicKey = issuerPublicKey
        self.nfts = {}  # Dictionnaire des NFTs

    def mint(self, ownerPublicKey, nftId):
        if nftId not in self.nfts:
            self.nfts[nftId] = ownerPublicKey
            print(f"NFT {nftId} frappé par {ownerPublicKey}")
        else:
            raise ValueError(f"NFT {nftId} déjà existant")

    def transfer(self, senderPublicKey, receiverPublicKey, nftId):
        if nftId in self.nfts and self.nfts[nftId] == senderPublicKey:
            self.nfts[nftId] = receiverPublicKey
            print(f"NFT {nftId} transféré de {senderPublicKey} à {receiverPublicKey}")
        else:
            raise ValueError("NFT inexistant ou propriété invalide")
"""

# Création du Smart Contract (définition)
NFT_contract = SmartContractDefinition(issuerPublicKey=walletProprietaire.publicKey, sourceCode=source_code, Ntoken=len(dico_nft))

# Signer le certificat
walletProprietaire.sign(NFT_contract)

# Instanciation du Smart Contract
NFT_instance = NFT_contract.instantiate_contract()

# Ajouter le Smart Contract au premier nœud (node1)
node1.new_certificate(NFT_contract) # Ajouter le smart contract au nœud

print(node1.blockchain.blockList)
print(node2.blockchain.blockList)
print(node3.blockchain.blockList)

Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x000001654E5746E0>
Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x000001654E5746E0>
Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x000001654E5746E0>


In [9]:
# On regarde si la période de minage est ouverte
NFT_contract.is_minting_open()

False

La période de minage est fermée donc les utilisateurs ne peuvent pas encore miner leur NFT. On peut quand même faire les tests :

In [10]:
try:
    assert NFT_contract.mint(ownerPublicKey=walletEdouard.publicKey, nftId="NFT1")
except ValueError as error:
    print(error)

La période de minage est fermée.


Il n'y a que le propriétaire du smart contract qui peut activer la période de minage. Ici c'est Alice !

In [11]:
NFT_contract.open_minting(walletProprietaire.publicKey, 1000)

Période de minage ouverte pendant 1000 secondes


In [12]:
assert NFT_contract.is_minting_open()
"Minting open!"

'Minting open!'

Maintenant qu'on a ouvert la periode de minage les utilisateurs peuvent miner des NFT

In [13]:
NFT_contract.mint(ownerPublicKey=walletEdouard.publicKey, nftId="NFT1")
print(NFT_contract.owners)

NFT NFT1 minée par 30820122300d06092a864886f70d01010105000382010f003082010a0282010100b389b877a0d5fbcc0db4ea7169f1bb2f7f06724d6310f4a1935261cd0d87d2aeacbaa1b0e351509927cd7908ef6c2055b5065bba15870e01009f8568af9c1bf6b34fb9339c2fd31fb6fb649ce91ad65ba85d8ac0b70e51b95572fa4297e2bba569f4f1a4ce27559b2e3f3675df875625532fee3f08379f493ad2e56271427e140453ff59a7166a7dd47dcee2689a9bb514405eac0fce6a54cfab69a5a85bb5df3d54fd6cf8782218fd5b08d06763bc4fcfc36df222cfb97c4ac73af533c44fa021074f3e7a6f0e28d991ee689a614eeea3585186628acd0e9121aa23fa4d719b818c395de06e3f2ab16116a5e42b2db4459ad72119f3176c76ab664f52fe15f30203010001
{'NFT1': '30820122300d06092a864886f70d01010105000382010f003082010a0282010100b389b877a0d5fbcc0db4ea7169f1bb2f7f06724d6310f4a1935261cd0d87d2aeacbaa1b0e351509927cd7908ef6c2055b5065bba15870e01009f8568af9c1bf6b34fb9339c2fd31fb6fb649ce91ad65ba85d8ac0b70e51b95572fa4297e2bba569f4f1a4ce27559b2e3f3675df875625532fee3f08379f493ad2e56271427e140453ff59a7166a7dd47dcee2689a9bb514405eac0fce6a54cfab69a5a85b

In [14]:
NFT_contract.showNFT(ownerNFT=walletEdouard.publicKey)

AttributeError: 'SmartContractDefinition' object has no attribute 'showNFT'

In [ ]:
NFT_contract.mint(ownerPublicKey=walletMatthieu.publicKey, nftId="NFT2")
NFT_contract.mint(ownerPublicKey=walletHugo.publicKey, nftId="NFT3")
NFT_contract.mint(ownerPublicKey=walletMilo.publicKey, nftId="NFT4")
NFT_contract.mint(ownerPublicKey=walletHenri.publicKey, nftId="NFT5")

On peut changer le propriétaire d'un NFT si c'est le propriétaire de ce dernier qui le demande

In [ ]:
assert NFT_contract.transfer(senderPublicKey=walletHenri.publicKey, receiverPublicKey=walletVincent.publicKey, nftId="NFT5")

In [ ]:
try :
    assert NFT_contract.transfer(senderPublicKey=walletHenri.publicKey, receiverPublicKey=walletVincent.publicKey, nftId="NFT5")
except ValueError as error:
    print(error)

In [ ]:
NFT_contract.showNFT(ownerNFT=walletMatthieu.publicKey)